## Reanalysis of Cuddy, Carney, Yap (2010)

[Data here](https://dataverse.harvard.edu/file.xhtml?persistentId=doi:10.7910/DVN/FMEGS6/U2QT5N&version=3.0#), not wgettable.

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("ccy-clean-data.csv")
# Drop actual outliers
df = df[df["inelig"] == "Analytic sample (keep)"]

expansiveGroup = df[df.hptreat == "High"]
contractiveGroup = df[df.hptreat == "Low"]

df.columns

Index(['id', 'inelig', 'ccydrop', 'cortm1v2', 'cortm2v2', 'cdiffv2',
       'testm1v2', 'testm2v2', 'tdiffv2', 'testoutv1', 'cortoutv1', 'anyoutv1',
       'testoutv2', 'cortoutv2', 'anyoutv2', 'pose1rate', 'pose2rate',
       'poseratem', 'saldiff', 'sal2manip', 'hptreat', 'female', 'age',
       'cort1a1', 'cort1a2', 'cort2a1', 'cort2a2', 'cortm1', 'cortm2', 'cdiff',
       'test1a1', 'test1a2', 'test2a1', 'test2a2', 'testm1', 'testm2', 'tdiff',
       'feelpower', 'incharge', 'powm', 'diceroll'],
      dtype='object')

In [3]:
def get_mean_and_sds(data, col) :
    stats = data[[col]].describe()
    
    return stats.loc["mean"][0], \
            stats.loc["std"][0]


expCortisolDiffMean, expCortisolDiffSd = get_mean_and_sds(expansiveGroup, "cdiffv2")
expTestosteroneDiffMean, expTestosteroneDiffSd = get_mean_and_sds(expansiveGroup, "tdiffv2")
expRiskProportion = expansiveGroup[expansiveGroup.diceroll == "Yes"].shape[0] / expansiveGroup.shape[0]

conCortisolDiffMean, conCortisolDiffSd = get_mean_and_sds(contractiveGroup, "cdiffv2")
conTestosteroneDiffMean, conTestosteroneDiffSd = get_mean_and_sds(contractiveGroup, "tdiffv2")
conRiskProportion = contractiveGroup[contractiveGroup.diceroll == "Yes"].shape[0] / contractiveGroup.shape[0]

In [4]:
def cohens_d(experimentalMean, controlMean, \
            experimentalSd, controlSd, \
            experimentalN, controlN, \
            correctForBias=True) :
    sd = pooled_sd(experimentalSd, controlSd)
    d = (experimentalMean - controlMean) / sd
    
    if correctForBias :
        N = experimentalN + controlN
        correction = ( (N - 3)/(N - 2.25) ) \
                        * np.sqrt( (N-2)/N )
        d = d * correction
    
    return d


def pooled_sd(sd1, sd2) :
    sumSquaredSigma = sd1**2 + sd2**2
    
    return np.sqrt(sumSquaredSigma/2)


# for proportions
def phi(p) :
    return 2 * np.arcsin(np.sqrt(p))


def cohens_h(p1, p2) :
    return phi(p1) - phi(p2)

In [5]:
cortisolD = cohens_d(expCortisolDiffMean, conCortisolDiffMean,
                    expCortisolDiffSd, conCortisolDiffSd, 
                    expansiveGroup.shape[0], contractiveGroup.shape[0], True)
cortisolD

-0.30388323292703984

In [6]:
testosteroneD = cohens_d(expTestosteroneDiffMean, conTestosteroneDiffMean,
                    expTestosteroneDiffSd, conTestosteroneDiffSd, 
                    expansiveGroup.shape[0], contractiveGroup.shape[0], True)
testosteroneD

0.35433781784037266

In [7]:
cohens_h(expRiskProportion, conRiskProportion)

0.6129820213361947